In [3]:
from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI
from dotenv import find_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langfuse.decorators import langfuse_context
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from dotenv import load_dotenv
import os

# 初始化 ERNIE 模型
ernie_model = QianfanChatEndpoint(
    qianfan_ak='nhW0x9Tocm3ZvyhldiBw6gen',
    qianfan_sk='DS9MyuRcxcyn5fCUyFUqBkvs7ksYxTuX',
    model='ERNIE-3.5-8K'
)

# 初始化 GPT 模型（默认是 GPT-3.5-Turbo）
gpt_model = ChatOpenAI(model="gpt-3.5-turbo")  # 确保 OPENAI_MODEL_NAME 定义为 "gpt-3.5-turbo"

# 配置模型选择逻辑，默认使用 ERNIE 模型
llm = gpt_model.configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="ernie_model",
    ernie=ernie_model
    # 其他模型可以类似添加，例如 claude=claude_model
)

class ModelInvoker:
    def __init__(self, llm):
        self.llm = llm

    def invoke_with_handler(self, input):
        langfuse_handler = langfuse_context.get_current_langchain_handler()
        ai_message = self.llm.invoke(input=input, config={"callbacks": [langfuse_handler], "llm": "ernie"})
        return ai_message

# 创建 ModelInvoker 实例
invoker = ModelInvoker(llm)

# 示例输入
input_text = "介绍你自己，包括你的生产商。"

# 调用 ERNIE 模型生成回复
response = invoker.invoke_with_handler(input_text)
print(response)


IndexError: list index out of range